In [1]:
import sys
sys.path.append('../src')

In [2]:
import pandas as pd
import numpy
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource, Scope
from energiapy.components.process import Process, VaryingProcess
from energiapy.components.material import Material
from energiapy.components.location import Location
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
from energiapy.components.transport import Transport
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.plot import plot_results, plot_scenario
from energiapy.plot.plot_results import CostY, CostX
from energiapy.model.solve import solve
from itertools import product
from energiapy.components.result import Result
from itertools import product
from matplotlib import rc
import matplotlib.pyplot as plt
from energiapy.model.solve import solve
from energiapy.plot.plot_results import CostY, CostX
from energiapy.components.location import Location
from energiapy.plot import plot_results, plot_scenario
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.model.bounds import CapacityBounds
from energiapy.utils.data_utils import get_data, make_henry_price_df, remove_outliers, load_results
from energiapy.conversion.photovoltaic import solar_power_output
from energiapy.conversion.windmill import wind_power_output
from energiapy.model.constraints.integer_cuts import constraint_block_integer_cut, constraint_block_integer_cut_min

In [3]:
from pyomo.environ import *

model = ConcreteModel()

# Sets and parameters
model.T = Set(initialize=[1, 2, 3])  # time steps
model.demand = Param(model.T, initialize={1: 100, 2: 120, 3: 110})
model.supply = Var(model.T, domain=NonNegativeReals)

# Binary indicators for penalty tiers
model.z1 = Var(model.T, domain=Binary)  # 0-10%
model.z2 = Var(model.T, domain=Binary)  # 10-20%
model.z3 = Var(model.T, domain=Binary)  # 20-30%
model.z4 = Var(model.T, domain=Binary)  # >30%

# Penalty variable
model.penalty_cost = Var(model.T, domain=NonNegativeReals)

# Only one tier active per time
def only_one_tier_rule(m, t):
    return m.z1[t] + m.z2[t] + m.z3[t] + m.z4[t] <= 1
model.one_tier = Constraint(model.T, rule=only_one_tier_rule)

# Auxiliary unmet variable
model.unmet = Var(model.T, domain=NonNegativeReals)

def unmet_demand_rule(m, t):
    return m.unmet[t] >= m.demand[t] - m.supply[t]
model.unmet_def = Constraint(model.T, rule=unmet_demand_rule)

# Penalty constraints
def penalty_definition(m, t):
    return m.penalty_cost[t] == (
        m.unmet[t] * 50 * m.z1[t] +
        m.unmet[t] * 80 * m.z2[t] +
        m.unmet[t] * 120 * m.z3[t] +
        m.unmet[t] * 200 * m.z4[t]
    )
model.penalty_eq = Constraint(model.T, rule=penalty_definition)

# Link unmet percentage to tiers
def tier_logic(m, t):
    demand_val = float(value(m.demand[t]))
    return [
        m.unmet[t] <= 0.10 * demand_val + (1 - m.z1[t]) * 1e5,
        m.unmet[t] >= 0.0001 * demand_val - (1 - m.z1[t]) * 1e5,
        m.unmet[t] <= 0.20 * demand_val + (1 - m.z2[t]) * 1e5,
        m.unmet[t] >= 0.10001 * demand_val - (1 - m.z2[t]) * 1e5,
        m.unmet[t] <= 0.30 * demand_val + (1 - m.z3[t]) * 1e5,
        m.unmet[t] >= 0.20001 * demand_val - (1 - m.z3[t]) * 1e5,
        m.unmet[t] >= 0.30001 * demand_val - (1 - m.z4[t]) * 1e5,
    ]
model.tier_constraints = ConstraintList()
for t in model.T:
    for rule in tier_logic(model, t):
        model.tier_constraints.add(rule)

In [13]:
from pyomo.environ import *

# Create model
model = ConcreteModel()

# Here I will have [1,365,24]
model.T = Set(initialize=[1, 2, 3])

# Demand will be daily and fixed for the entire 365 time periods
demand_data = {1: 100, 2: 120, 3: 110}
model.demand = Param(model.T, initialize=demand_data)

# We create hydrogen through different processes in the ESCAPE case study
model.supply = Var(model.T, domain=NonNegativeReals)

# Unmet demand
model.unmet = Var(model.T, domain=NonNegativeReals)

# Binary variables for penalty tiers (only one active)
# IF the study gets to large, I will reduce the number of tiers
model.z1 = Var(model.T, domain=Binary)  # 0–10%
model.z2 = Var(model.T, domain=Binary)  # 10–20%
model.z3 = Var(model.T, domain=Binary)  # 20–30%
model.z4 = Var(model.T, domain=Binary)  # >30%

# Penalty cost per time period
model.penalty_cost = Var(model.T, domain=NonNegativeReals)

# Objective: total penalty
model.total_penalty = Var(domain=NonNegativeReals)

# Constraint: unmet demand
def unmet_def_rule(m, t):
    return m.unmet[t] >= m.demand[t] - m.supply[t]
model.unmet_def = Constraint(model.T, rule=unmet_def_rule)

# Constraint: only one tier active per period
def one_tier_rule(m, t):
    return m.z1[t] + m.z2[t] + m.z3[t] + m.z4[t] == 1
model.only_one_tier = Constraint(model.T, rule=one_tier_rule)

# Tier constraints: determine which tier the unmet % falls into
bigM = 1e5
model.tier_logic = ConstraintList()
for t in model.T:
    D = float(value(model.demand[t]))
    # Tier 1: 0–10%
    model.tier_logic.add(model.unmet[t] <= 0.10 * D + (1 - model.z1[t]) * bigM)
    model.tier_logic.add(model.unmet[t] >= 0 * D - (1 - model.z1[t]) * bigM)
    # Tier 2: 10–20%
    model.tier_logic.add(model.unmet[t] <= 0.20 * D + (1 - model.z2[t]) * bigM)
    model.tier_logic.add(model.unmet[t] >= 0.10001 * D - (1 - model.z2[t]) * bigM)
    # Tier 3: 20–30%
    model.tier_logic.add(model.unmet[t] <= 0.30 * D + (1 - model.z3[t]) * bigM)
    model.tier_logic.add(model.unmet[t] >= 0.20001 * D - (1 - model.z3[t]) * bigM)
    # Tier 4: >30%
    model.tier_logic.add(model.unmet[t] >= 0.30001 * D - (1 - model.z4[t]) * bigM)

# Penalty cost: full unmet * tier rate - I want to do the absolute penalty cost
def penalty_cost_rule(m, t):
    return m.penalty_cost[t] == (
        m.unmet[t] * 50 * m.z1[t] +
        m.unmet[t] * 80 * m.z2[t] +
        m.unmet[t] * 120 * m.z3[t] +
        m.unmet[t] * 200 * m.z4[t]
    )
model.penalty_eq = Constraint(model.T, rule=penalty_cost_rule)

# Total penalty
def total_penalty_rule(m):
    return m.total_penalty == sum(m.penalty_cost[t] for t in m.T)
model.total_penalty_eq = Constraint(rule=total_penalty_rule)

# Objective: minimize total penalty
model.obj = Objective(expr=model.total_penalty, sense=minimize)

# Solve
solver = SolverFactory('gurobi')  # Replace with 'gurobi' or 'cbc' if needed
solver.solve(model, tee=True)

# Print results
print("\nResults:")
'stupid formulation that I used to do in APO to print specific parts'
# for t in model.T:
#     print(f"Time {t}:")
#     print(f"  Demand: {value(model.demand[t])}")
#     print(f"  Supply: {value(model.supply[t]):.2f}")
#     print(f"  Unmet: {value(model.unmet[t]):.2f}")
#     print(f"  Penalty: €{value(model.penalty_cost[t]):.2f}")
#     tier = "?"
#     if value(model.z1[t]) > 0.5: tier = "0–10%"
#     elif value(model.z2[t]) > 0.5: tier = "10–20%"
#     elif value(model.z3[t]) > 0.5: tier = "20–30%"
#     elif value(model.z4[t]) > 0.5: tier = ">30%"
#     print(f"  Tier: {tier}")
print(f"\nTotal Penalty: €{value(model.total_penalty):.2f}")

Read LP format model from file C:\Users\MARCOP~1\AppData\Local\Temp\tmp0hn754zu.pyomo.lp
Reading time = 0.00 seconds
x1: 28 rows, 22 columns, 64 nonzeros
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 28 rows, 22 columns and 64 nonzeros
Model fingerprint: 0x67a7199e
Model has 3 quadratic constraints
Variable types: 10 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [5e+01, 2e+02]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 25 rows and 17 columns
Presolve time: 0.00s
Presolved: 7 rows, 9 columns, 24 nonzeros
Variable types: 5 continuous, 4 integer (4 binary)
Found heuristic solution: objective 6600.2200


Results:

Total Penalty: €0.00
